In [1]:
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix, vstack, hstack, save_npz, load_npz, block_diag, identity, random
from scipy.sparse.linalg import inv, spsolve, splu
from scipy.linalg import lu, lu_factor, lu_solve
from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Pool, shared_memory
import matplotlib.pyplot as plt
import time
import cProfile
import pstats 
import csv
import os 
from tqdm import tqdm
import gdown

In [2]:
def lower_block_bidiagonal_nonsingular(n_blocks, block_size):
    """
    Generate a nonsingular sparse lower block bidiagonal matrix in CSR format.

    Parameters:
        n_blocks (int): Number of diagonal blocks.
        block_size (int): Size of each square block.

    Returns:
        scipy.sparse.csr_matrix: The resulting nonsingular sparse matrix.
        numpy.ndarray: The corresponding RHS vector.
    """
    N = n_blocks * block_size  # Total size of the matrix
    data, row_indices, col_indices = [], [], []

    # Generate diagonal (B_i) and lower diagonal (L_i) blocks
    for i in range(n_blocks):
        row_offset = i * block_size
        col_offset = i * block_size

        # Ensure nonzero entries in the main diagonal block (B_i)
        block_main = np.random.rand(block_size, block_size) + np.eye(block_size)  # Make B_i non-singular
        for r in range(block_size):
            for c in range(block_size):
                val = block_main[r, c]
                data.append(val)
                row_indices.append(row_offset + r)
                col_indices.append(col_offset + c)

        # Lower block (L_i), ensuring nonzero entries
        if i < n_blocks - 1:
            row_offset = (i + 1) * block_size
            col_offset = i * block_size
            block_lower = np.random.rand(block_size, block_size) + 0.5*np.eye(block_size) # Random values ensure nonzero entries

            for r in range(block_size):
                for c in range(block_size):
                    val = block_lower[r, c]
                    data.append(val)
                    row_indices.append(row_offset + r)
                    col_indices.append(col_offset + c)

    # Create sparse CSR matrix
    sparse_matrix = csr_matrix((data, (row_indices, col_indices)), shape=(N, N))

    # Generate a random RHS vector (column vector)
    rhs_vector = np.random.rand(N, 1)  # Nx1 dense vector

    return sparse_matrix, rhs_vector

In [5]:
block_size = 4
number_of_processors = 4

#k_list = [4,5,6,7,8,9,10,11,12,13,14,15,16]
k_list = [18]
for k in tqdm(k_list):
    n = int(number_of_processors*2**k)
    number_of_blocks = n + 1
    print(k)
    print(number_of_blocks)
    M, f = lower_block_bidiagonal_nonsingular(number_of_blocks, block_size)
    x = spsolve(M,f)
    save_folder = f"Samples_to_test"
    save_npz(f"n{number_of_blocks}_b{block_size}_mat.npz",M)
    np.save(f"n{number_of_blocks}_b{block_size}_rhs.npy",f)
    np.save(f"n{number_of_blocks}_b{block_size}_sol.npy",x)

  0%|          | 0/1 [00:00<?, ?it/s]

18
1048577


  0%|          | 0/1 [01:42<?, ?it/s]


RuntimeError: SUPERLU_MALLOC fails for buf in intCalloc() at line 173 in file ../scipy/sparse/linalg/_dsolve/SuperLU/SRC/memory.c


In [4]:
def create_full_permutation_matrix(m, block_size):
    num_blocks = m // block_size
    perm_order = []

    # First, append all odd-indexed blocks
    for i in range(0, num_blocks, 2):
        perm_order.append(i)

    # Then, append all even-indexed blocks
    for i in range(1, num_blocks, 2):
        perm_order.append(i)

    data, rows, cols = [], [], []

    for new_index, old_index in enumerate(perm_order):
        start_new = new_index * block_size
        start_old = old_index * block_size
        for i in range(block_size):
            data.append(1)              
            rows.append(start_new + i)  
            cols.append(start_old + i)  

    return csr_matrix((data, (rows, cols)), shape=(m, m))

def placeholder_name(M, f, block_size : int, processors : int):
    """ 
    Performs Block Cyclic Reduction (BCR) in parallel for solving lower block bidiagonal systems.

    Parameters:
    -----------
    M : scipy.sparse.csr_matrix or numpy.ndarray
        The coefficient matrix of size (N, N), where N = (n+1) * block_size.
        Must be a square lower block bidiagonal matrix.

    f : numpy.ndarray
        The right-hand side (RHS) vector of size (N, 1), corresponding to Mx = f.

    block_size : int
        The size of each block in the matrix.

    processors : int
        The number of processors used for parallel block cyclic reduction.

    Returns:
    --------
    x : numpy.ndarray
        The solution vector of size (N, 1) satisfying Mx = f.
    """
    N, L = M.shape
    assert N == L,  f"M must be sqaure but has dimensions {N}x{L}"
    n = (N - 1) // block_size
    assert n % processors == 0, f"M must have size (n+1)*block_size x (n+1)*block_size, where n = p * 2**k. p is not a multiple of n."
    nbyp = n // processors 
    assert ((nbyp & (nbyp-1) == 0) and nbyp != 0), f"M must have size (n+1)*block_size x (n+1)*block_size, where n = p * 2**k. n/p is not a power of two."
    number_of_steps = int(np.log2(nbyp)) # Number of steps in the forward step and backward step for each processor
    
    # Divide among the processors
    M_k_list = []
    f_k_list = []
    B_k_s_list = []
    A_k_s_list = []
    f_k_s_list = []
    
    for i in range(processors):
        # Perform the forward step
        M_copy = M[
            block_size*(nbyp*i+1):block_size*(nbyp*(i+1)+1),
            block_size*(nbyp*i):block_size*(nbyp*(i+1)+1)
        ]
        f_copy = f[
            block_size*(nbyp*i+1):block_size*(nbyp*(i+1)+1)
        ]
        M_k, f_k, B_k_s, A_k_s, f_k_s = forward_placeholder(M_copy, f_copy, block_size, processors, [], [], [])
        #print("One Forward complete ##############################")

        # Store the results for inter-processor communication
        M_k_list.append(M_k)
        f_k_list.append(f_k)

        # Store the results for the backward step
        B_k_s_list.append(B_k_s)
        A_k_s_list.append(A_k_s)
        f_k_s_list.append(f_k_s)

    
    x0 = spsolve(M[:block_size,:block_size],f[:block_size]) # The master of all processors
    base_case_x = [x0]

    # Only serial part of the algorithm
    for i in range(processors):
        B_k = M_k_list[i][:,:block_size]
        A_k = M_k_list[i][:,block_size:]
        f_k = f_k_list[i]
        x_next = spsolve(A_k,f_k.flatten()-B_k@base_case_x[i])
        base_case_x.append(x_next)
    
    final_x = np.array([])
    
    # Perform the backward step
    for i in range(processors):
        B_k_s = B_k_s_list[i] 
        A_k_s = A_k_s_list[i] 
        f_k_s = f_k_s_list[i] 

        x_for_current_processor = backward_placeholder(B_k_s, A_k_s, f_k_s, base_case_x[i:i+2], number_of_steps, block_size, processors)
        final_x = np.concatenate((final_x, x_for_current_processor))

    final_x = np.concatenate((final_x, base_case_x[-1]))
    return final_x
        
def forward_placeholder(M, f, block_size : int, processors : int, B_s = [], A_s = [], f_s = []):
    m = M.shape[0]//block_size
    if m == 1:
        return M,f,B_s,A_s,f_s
    
    
    M_next = csr_matrix((block_size*m//2,block_size*m//2+block_size))
    f_next = np.zeros(block_size*m//2)
    I = np.eye(block_size)
    # Do one step
    for i in range(0,m,2):
        # Extract block elements from input
        B1 = M[
            block_size*i:block_size*(i+1), 
            block_size*i:block_size*(i+1)
            ] 
        A1 = M[
            block_size*i:block_size*(i+1), 
            block_size*(i+1):block_size*(i+2)
            ] 
        B2 = M[
            block_size*(i+1):block_size*(i+2),
            block_size*(i+1):block_size*(i+2)
            ]
        A2 = M[
            block_size*(i+1):block_size*(i+2),
            block_size*(i+2):block_size*(i+3)
            ]
        f1 = f[block_size*i:block_size*(i+1)]
        f2 = f[block_size*(i+1):block_size*(i+2)]
        
        # print(f"B1: \n{B1.toarray()} ")
        # print(f"A1: \n{A1.toarray()} ")
        # print(f"B2: \n{B2.toarray()}")
        # print(f"A2: \n{A2.toarray()}")
        # print("\n")

        # Store the values for the backward step
        B_s.append(B1)
        A_s.append(A1)
        f_s.append(f1)

        # Convert blocks to dense arrays if they aren’t already.
        # B1_dense = B1.toarray() if hasattr(B1, "toarray") else B1
        # A1_dense = A1.toarray() if hasattr(A1, "toarray") else A1
        # B2_dense = B2.toarray() if hasattr(B2, "toarray") else B2
        # A2_dense = A2.toarray() if hasattr(A2, "toarray") else A2
        # f1_dense = f1.toarray() if hasattr(f1, "toarray") else f1
        # f2_dense = f2.toarray() if hasattr(f2, "toarray") else f2

        # print(f"B1: {B1.toarray()}")
        # print(f"A1: {A1.toarray()}")
        # print(f"B2: {B2.toarray()}")
        # print(f"A2: {A2.toarray()}")
        # print(f"f1: {f1}")
        # print("\n")

        # # Use LU factorization with pivoting to "solve" for the necessary inverses.
        # lu_A1, piv_A1 = lu_factor(A1_dense)
        # lu_B2, piv_B2 = lu_factor(B2_dense)

        # # Instead of explicitly forming inverses, we solve linear systems:
        # new_B1 = lu_solve((lu_A1, piv_A1), B1_dense)
        # new_A1 = -lu_solve((lu_B2, piv_B2), A2_dense)
        # new_f1 = lu_solve((lu_A1, piv_A1), f1_dense) - lu_solve((lu_B2, piv_B2), f2_dense)
        A1_inv = spsolve(A1, I)
        B2A1_inv = B2@A1_inv
        new_B1 = B2A1_inv@B1
        new_A1 = -A2
        new_f1 = B2A1_inv@f1 - f2
        # Set the new values to obtain a reduced system of half the original size
        j = i//2
        M_next[block_size*j:block_size*(j+1),block_size*j:block_size*(j+1)] = new_B1
        M_next[block_size*j:block_size*(j+1),block_size*(j+1):block_size*(j+2)] = new_A1
        f_next[block_size*j:block_size*(j+1)] = new_f1.flatten()

    # Recursively apply the same procedure
    return forward_placeholder(M_next,f_next,block_size,processors,B_s,A_s,f_s)

def backward_placeholder(B_s, A_s, f_s, x_s, number_of_steps : int, block_size : int, processors : int):
    x_result = x_s[0]
    power = 0
    for i in range(number_of_steps-1,-1,-1):
        j = -(2**power - 1) if power > 0 else None
        k = -(2**(power+1) - 1)

        B = B_s[k:j]
        A = A_s[k:j]
        f = f_s[k:j]

        A_for_solve = block_diag(A, format='csr')
        B_for_solve = block_diag(B, format='csr')
        f_for_solve = np.concatenate(f).flatten()
        x_for_solve = x_result.copy()   

        x_new = spsolve(A_for_solve,f_for_solve - B_for_solve@x_for_solve)

        x_result = np.concatenate((x_result,x_new))
        Q = create_full_permutation_matrix(x_result.shape[0], block_size)
        x_result = Q.T@x_result
        power += 1
    return x_result

In [7]:
save_folder = "Samples_to_test"
n_blocks = int(number_of_processors*2**k_list[7]) + 1
print(n_blocks)
M,f,x = load_npz(f"{save_folder}/n{n_blocks}_b{number_of_processors}_mat.npz"), np.load(f"{save_folder}/n{n_blocks}_b{number_of_processors}_rhs.npy"), np.load(f"{save_folder}/n{n_blocks}_b{number_of_processors}_sol.npy")

8193


In [8]:
start = time.time()
x_sol = placeholder_name(M,f,block_size=block_size,processors=number_of_processors)
end = time.time()
print(f"Time taken: {end-start} seconds")
print("Error,", np.linalg.norm(x - x_sol))

Time taken: 19.89903688430786 seconds
Error, 1.6277922032608407e-11


In [7]:
import numpy as np
from scipy.sparse import issparse
from scipy.sparse.linalg import svds

def compute_condition_number(A, norm=2):
    """
    Compute the condition number of a square matrix A.
    
    Parameters:
        A : numpy.ndarray or scipy.sparse matrix
            The square matrix whose condition number is to be computed.
        norm : {2, -1, -np.inf, 'fro'}, optional
            The matrix norm to use. For norm=2, the condition number is computed via
            singular values. For other norms, np.linalg.cond is used on a dense array.
            (Default is 2, which is common for spectral norm.)
    
    Returns:
        cond_number : float
            The computed condition number of A.
    """
    # For sparse matrices, if the matrix is small, convert to dense;
    # otherwise, use a sparse SVD for the 2-norm.
    if issparse(A):
        n, m = A.shape
        if n != m:
            raise ValueError("Matrix must be square.")
        
        # Use dense conversion if the matrix is moderate in size.
        # Adjust the threshold as needed.
        if n < 500:
            A_dense = A.toarray()
            cond_number = np.linalg.cond(A_dense, p=norm)
        else:
            # For norm=2, estimate the largest and smallest singular values.
            if norm == 2:
                # Compute the largest singular value.
                sigma_max = svds(A, k=1, which='LM', return_singular_vectors=False)[0]
                # Compute the smallest singular value.
                sigma_min = svds(A, k=1, which='SM', return_singular_vectors=False)[0]
                cond_number = sigma_max / sigma_min
            else:
                # For other norms, convert to dense (or implement another method).
                A_dense = A.toarray()
                cond_number = np.linalg.cond(A_dense, p=norm)
    else:
        # Dense case.
        cond_number = np.linalg.cond(A, p=norm)
    
    return cond_number

print(compute_condition_number(M))

297.20652354268515


In [10]:
import numpy as np
from scipy.sparse import issparse

def compute_lower_bidiagonal_block_condition_numbers(M, block_size, norm=2):
    """
    Given a lower block-bidiagonal matrix M with blocks of size `block_size`, compute the condition numbers 
    for each block in the following manner:
    
        Row 0: Only A_0 exists, so its condition number is computed.
        For each row i >= 1:
            - B_i is the off-diagonal block (located in columns [(i-1)*block_size, i*block_size])
            - A_i is the diagonal block (located in columns [i*block_size, (i+1)*block_size])
    
    Parameters:
        M : numpy.ndarray or scipy.sparse matrix
            The full square matrix of size (n_blocks * block_size) x (n_blocks * block_size) having the structure:
            
                [ A_0       0      0    ... ]
                [ B_1      A_1     0    ... ]
                [  0       B_2    A_2   ... ]
                [ ...      ...    ...   ... ]
            
        block_size : int
            The size of each square block.
        norm : {2, -1, -np.inf, 'fro'}, optional
            The matrix norm to use when computing the condition number (default is 2, the spectral norm).
    
    Returns:
        cond_dict : dict
            A dictionary where:
              - Key 0 maps to the condition number of A_0.
              - For each i >= 1, key i maps to a tuple (cond(B_i), cond(A_i)).
    """
    # Convert M to a dense array if it's sparse.
    if issparse(M):
        M_dense = M.toarray()
    else:
        M_dense = M

    n, m = M_dense.shape
    if n != m:
        raise ValueError("Matrix must be square.")
    if n % block_size != 0:
        raise ValueError("Matrix dimensions must be an integer multiple of block_size.")
    
    nblocks = n // block_size
    cond_dict = {}
    
    # Block row 0: Only the diagonal block A_0 is present.
    A0 = M_dense[0:block_size, 0:block_size]
    cond_dict[0] = np.linalg.cond(A0, p=norm) if not np.allclose(A0, 0) else np.inf
    
    # For each subsequent block row i (i >= 1), compute:
    # - cond(B_i) for the off-diagonal block from the previous column block.
    # - cond(A_i) for the diagonal block.
    for i in range(1, nblocks):
        Bi = M_dense[i*block_size:(i+1)*block_size, (i-1)*block_size:i*block_size]
        Ai = M_dense[i*block_size:(i+1)*block_size, i*block_size:(i+1)*block_size]
        cond_Bi = np.linalg.cond(Bi, p=norm) if not np.allclose(Bi, 0) else np.inf
        cond_Ai = np.linalg.cond(Ai, p=norm) if not np.allclose(Ai, 0) else np.inf
        cond_dict[i] = (cond_Bi, cond_Ai)
    
    return cond_dict

cond_dict = compute_lower_bidiagonal_block_condition_numbers(M, block_size)

In [28]:
import numpy as np
from scipy.sparse import csr_matrix, issparse

def print_nonzero_block_rows(M, block_size):
    """
    Print the nonzero block rows of a CSR matrix M.

    The matrix M is assumed to be partitioned into block rows of height `block_size`.
    For example, if M has shape (n, m) with n = n_blocks * block_size, then each
    block row consists of rows [i*block_size : (i+1)*block_size].

    Parameters:
      M : csr_matrix or any matrix convertible to dense (if needed)
          The input matrix.
      block_size : int
          The number of rows in each block.

    This function prints each block row (converted to a dense array) that has at least
    one nonzero element.
    """
    # If M is sparse, we assume it's in CSR format; otherwise, convert it.
    if not issparse(M):
        M = csr_matrix(M)

    n, _ = M.shape
    if n % block_size != 0:
        print("Warning: The number of rows is not an integer multiple of block_size.")
    
    num_block_rows = n // block_size
    row_start = 0
    row_end = 1
    col_start = 0
    col_end = 1
    for block_index in range(num_block_rows):
        A = M[block_index*block_size:(block_index+1)*block_size, block_index*block_size:(block_index+1)*block_size]
        res1 = spsolve(A,np.eye(block_size))
        print(res1)
        
          # Empty matrix if no off-diagonal block
        if block_index != 0:
            B = M[(block_index)*block_size:(block_index+1)*block_size, (block_index-1)*block_size:(block_index)*block_size]
            res2 = spsolve(B,np.eye(block_size))
            print(res2,"\n")
        else:
            B = csr_matrix((block_size, block_size))
        #print(f"A_{block_index}: \n",A.toarray())
        #print(f"B_{block_index}: \n",B.toarray(),"\n")
        # save all A's and B's to one .txt file with easy to read format such that A_0, B_0, A_1, B_1, ...
        # with open("A_B_blocks.txt", "a") as f:
        #     f.write(f"B_{block_index}: \n")
        #     f.write(str(B.toarray()))
        #     f.write("\n")
        #     f.write(f"A_{block_index}: \n")
        #     f.write(str(A.toarray()))
            
            
            # f.write("\n\n")


# With block_size=2, we have two block rows: rows 0-1 and rows 2-3.
print_nonzero_block_rows(M, block_size=4)


[[ 0.9591832  -0.37027355 -0.52384946  0.2566584 ]
 [ 0.13703053  0.86736092 -0.20603349 -0.66015064]
 [-0.48010181 -0.04684315  0.97464301  0.00411903]
 [-0.6664533   0.19752411  0.22528916  0.68060908]]
[[ 1.32731478 -0.40269314  0.2141266  -0.80529709]
 [-0.19896861  1.34443433 -0.45221739 -0.37487858]
 [-0.01696623 -0.3188554   0.6988002   0.09112909]
 [-0.64573353 -0.68225848 -0.30696869  1.68782664]]
[[ -3.80872872   6.04086724  -3.49196719   0.28653844]
 [-41.05709306  48.10053451 -26.05691205   2.20051504]
 [ 50.11314563 -59.52253352  33.6007555   -3.00669035]
 [  4.64666442  -6.02467365   3.27268344   0.49857342]] 

[[ 0.81119996 -0.37734878 -0.74109342  0.26059835]
 [-0.12373811  0.82015736  0.13714021 -0.19764915]
 [ 0.06417305  0.13834978  0.95473993 -0.60027666]
 [-0.59736189 -0.42070465  0.27393742  1.0520907 ]]
[[ 1.20833369  0.29730396 -0.19344599 -1.20669316]
 [-6.96227079  4.54052003 -1.71616909  8.17991036]
 [ 4.363609   -2.65736275  2.09014811 -6.39069038]
 [ 0.2076